In [582]:
%reload_ext autoreload
%autoreload 2

In [583]:
import io
import pymongo
from IPython import display

from bokeh.io import output_notebook, show
from bokeh.palettes import all_palettes
from bokeh.layouts import gridplot

import climatools.lblnew.bestfit_params as bestfits
from climatools.cliradlw.dataset import load_output_file
from climatools.cliradlw.utils import mapband_new2old
import climatools.html.html as climahtml
from climatools.plot.plot import *


output_notebook()

Loading BokehJS ...

In [584]:
client = pymongo.MongoClient('localhost', 27017)

In [585]:
'''
Edit here to start with!
'''

GAS = 'co2'
BAND = 4
CONC = 400e-6
ATMPRO = 'mls'


def query_clirad():
    q = {'molecule': {GAS: CONC},
         'atmpro': ATMPRO,
         'band': [BAND],
         'commitnumber': 'a6676f4'}
    return {f'param.{n}': v for n, v in q.items()}



def query_lblnewbestfit():
    q = {'molecule': GAS,
         'conc': CONC,
         'atmpro': ATMPRO,
         'band': mapband_new2old()[BAND],
         'commitnumber': 'a22ab94'}
    return {f'param.{n}': v for n, v in q.items()}


def query_lblnewbestfit1():
    q = {'molecule': GAS,
         'conc': CONC,
         'atmpro': ATMPRO,
         'band': mapband_new2old()[BAND],
         'commitnumber': 'a22ab94',
         'wgt': [(0, 0.6, 0.5, 0.7, 0.75), (0.7, 0.7)]}
    return {f'param.{n}': v for n, v in q.items()}

In [586]:
DOC_CLIRAD = client.cliradnew.lw.find_one(query_clirad())
DS_FLUX = load_output_file(io.StringIO(DOC_CLIRAD['output_flux']))
DS_COOL = load_output_file(io.StringIO(DOC_CLIRAD['output_coolr']))

In [587]:
DOC_LBLNEW = client.lblnew.bestfit_lw.find_one(query_lblnewbestfit())
DS_FLUX_CRD = load_output_file(io.StringIO(DOC_LBLNEW['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(DOC_LBLNEW['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(DOC_LBLNEW['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(DOC_LBLNEW['output_wcoolrg']))

In [588]:
DOC_LBLNEW1 = client.lblnew.bestfit_lw.find_one(
    query_lblnewbestfit1())
DS_FLUX_CRD1 = load_output_file(
    io.StringIO(DOC_LBLNEW1['output_fluxg']))
DS_COOL_CRD1 = load_output_file(
    io.StringIO(DOC_LBLNEW1['output_coolrg']))
DS_FLUX_WGT1 = load_output_file(
    io.StringIO(DOC_LBLNEW1['output_wfluxg']))
DS_COOL_WGT1 = load_output_file(
    io.StringIO(DOC_LBLNEW1['output_wcoolrg']))

In [589]:
def show_html(s):
    display.display(display.HTML(s))
    
def show_markdown(s):
    display.display(display.Markdown(s))

In [590]:
def script_print_param():
    docs = [DOC_LBLNEW, DOC_LBLNEW1]
    params = [doc['param'] for doc in docs]
    
#    param_bestfit = bestfits.kdist_params(molecule=GAS,
#                                          band=mapband_new2old()[BAND])    
#    params.append(param_bestfit)
    labels = ['lblnew', "lblnew wgt-adjusted"]
    
    df = pd.DataFrame(columns=labels)
    
    for label, param in zip(labels, params):
        for n, v in param.items():
            df.loc[n, label] = str(v)
            
    df.index.name = 'Input parameter'
    df.columns.name = 'Run'
    
    display.display(df)
    show_markdown('*TABLE.* Input parameters of runs considered here.')

In [591]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']



def pltdata_cool():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6},
            {'label': 'WGT igg=10 wgt-adjusted',
             'srs': fmt_cool(DS_COOL_WGT1.sel(igg=10)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6}]
    return data



def plt_cool(pltdata=None):
    '''
    Plot cooling rate profiles on linear
    and log pressure axes.
    
    Parameters
    ----------
    pltdata: dict
        Plot data, including colour, line attributes, etc.
    everything: bokeh.models.layouts.Column
        Cooling rate profile plot.
    '''
    plin = plt_vert_profile_bokeh(pltdata=pltdata, 
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200)) 
    plog.plot_width, plog.plot_height = 400, 500
    
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
    

def script_plt_cool():
    '''
    Script to plot the cooling rate profiles.
    '''
    data = pltdata_cool()
    p = plt_cool(pltdata=data)
    show(p)
    display.display(
        display.Markdown(f'*FIGURE.* Cooling rates. Summed over g.'))
    

In [592]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds:
        if ds['igg'].shape == (1,):
            ds = ds.squeeze('igg').drop('igg')
        elif ds['igg'].shape == ():
            ds = ds.drop('igg')
        
    if 'g' in ds:
        if ds['g'].shape == (1,):
            ds = ds.squeeze('g').drop('g')
        elif ds['g'].shape == ():
            ds = ds.drop('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df



def tbdata_flux(g=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    ilevels = [0, tropopauses[ATMPRO], -1]
    
    if g == None:
        ds_flux_crd = DS_FLUX_CRD.sum('g')
        ds_flux_wgt = DS_FLUX_WGT.sum('g')
        ds_flux_wgt1 = DS_FLUX_WGT1.sum('g')
    else:
        ds_flux_crd = DS_FLUX_CRD.sel(g=g)
        ds_flux_wgt = DS_FLUX_WGT.sel(g=g)
        ds_flux_wgt1 = DS_FLUX_WGT1.sel(g=g)
        
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(ds_flux_wgt.sel(igg=[10]), ilevels=ilevels)},
           {'label': 'WGT igg=10 wgt-adjusted',
            'df': fmt_flux(ds_flux_wgt1.sel(igg=[10]), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data



def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown((f"*TABLE.* Flux. {d['label']}"))
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(("*TABLE.* Flux difference."
                       f"({d['label']}) - ({d0['label']})"))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        

        
def script_tb_flux():
    
    for g in DS_FLUX_CRD.coords['g']:
        show_html(climahtml.getHTML_idanchor(f"g={int(g)} Flux Comparion"))
        show_markdown(f"### g={int(g)} Flux Comparison")
        data = tbdata_flux(g=int(g))
        tb_flux(tbdata=data)
        
    show_html(climahtml.getHTML_idanchor(f"g-total Flux Comparison"))    
    show_markdown(f"### g-total Flux Comparison")
    data = tbdata_flux()
    tb_flux(tbdata=data)
    


In [593]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'WGT igg=10', 'WGT igg=10 wgt-adjusted']
    dss = [DS_COOL_CRD, 
           DS_COOL_WGT.sel(igg=10), 
           DS_COOL_WGT1.sel(igg=10)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [594]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'WGT igg=10', 'WGT igg=10 wgt-adjusted']
    dss = [DS_FLUX_CRD.sum('g'), 
           DS_FLUX_WGT.sel(igg=10).sum('g'), 
           DS_FLUX_WGT1.sel(igg=10).sum('g')]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [595]:
def script():
    
    title = f'### {MOLECULE}{BAND} wgt adjustments'
    s_input = 'Input Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown(title)
    show_markdown('''
    Adjust wgt in co2 band04
    wgt[4] -= .05
    wgt[5] -= .1
    ''')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
#    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    for g in DS_FLUX_WGT.coords['g']:
        show_html(climahtml.getHTML_hrefanchor(f"g={int(g)} Flux Comparion"))
    show_html(climahtml.getHTML_hrefanchor(f"g-total Flux Comparison"))  
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    script_print_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cool()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [596]:
script()

### co24 wgt adjustments


    Adjust wgt in co2 band04
    wgt[4] -= .05
    wgt[5] -= .1
    

### Table of Contents

# Input Parameters

Run,lblnew,lblnew wgt-adjusted
Input parameter,,
atmpro,mls,mls
band,3b,3b
commitnumber,a22ab94,a22ab94
conc,0.0004,0.0004
dv,0.001,0.001
klin,0,0
molecule,co2,co2
ng_adju,"[0, 0]","[0, 0]"
ng_refs,"[5, 2]","[5, 2]"


...

*TABLE.* Flux difference.(WGT igg=10 wgt-adjusted) - (CRD)

# Cooling Rate by Layer

CRD            WGT igg=10            WGT igg=10 wgt-adjusted  \
            layer     coolrg      layer     coolrg                   layer   
pressure                                                                     
0.000312        1  76.722160          1  39.257572                       1   
0.000750        2  40.961180          2  34.206176                       2   
0.001052        3  33.041424          3  31.957575                       3   
0.001476        4  26.177150          4  28.878260                       4   
0.002070        5  20.453586          5  24.898122                       5   
0.002904        6  15.887011          6  20.205158                       6   
0.004074        7  12.355441          7  15.244785                       7   
0.005714        8   9.670386          8  10.682080                       8   
0.008015        9   7.632119          9   7.175222                       9   
0.011243       10   6.059189         10   5.016015                      10   
0.015771       11   4.800884         11   3.937337                      11   
0.022122       12   3.733240         12   3.354814                      12   
0.031031       13   2.741506         13   2.787181                      13   
0.043528       14   1.535104         14   1.820727                      14   
0.061057       15   0.621270         15   0.946735                      15   
0.085645       16   1.009105         16   1.174465                      16   
0.120136       17   1.978014         17   1.895884                      17   
0.168516       18   2.672370         18   2.331050                      18   
0.236378       19   3.351253         19   2.836185                      19   
0.331549       20   4.134225         20   3.719706                      20   
0.465100       21   5.128998         21   5.067896                      21   
0.652400       22   6.668315         22   6.928824                      22   
0.915100       23   8.412267         23   8.612716                      23   
1.283650       24   8.111907         24   7.923915                      24   
1.800600       25   6.297593         25   5.820153                      25   
2.525700       26   4.914451         26   4.706752                      26   
3.542800       27   3.934145         27   4.173922                      27   
4.969550       28   3.201959         28   3.474391                      28   
6.970850       29   2.668698         29   2.674056                      29   
9.778100       30   2.244095         30   2.188001                      30   
13.715850      31   1.822880         31   2.018952                      31   
19.239350      32   1.385437         32   1.705318                      32   
26.987250      33   1.051152         33   1.211673                      33   
37.855300      34   0.824001         34   0.818805                      34   
53.100050      35   0.576221         35   0.618111                      35   
73.887500      36   0.318069         36   0.429466                      36   
97.662500      37   0.177026         37   0.246796                      37   
121.437500     38   0.131797         38   0.138174                      38   
145.212500     39   0.032605         39  -0.003878                      39   
168.987500     40  -0.100428         40  -0.158179                      40   
192.762500     41  -0.096020         41  -0.145793                      41   
216.537500     42   0.023673         42  -0.003498                      42   
240.312500     43   0.075846         43   0.056112                      43   
264.087500     44   0.091997         44   0.071633                      44   
287.862500     45   0.098112         45   0.078186                      45   
311.637500     46   0.099724         46   0.082843                      46   
335.412500     47   0.098896         47   0.086509                      47   
359.187500     48   0.096725         48   0.088980                      48   
382.962500     49   0.093992        

# Fluxes by Layer

CRD                       WGT igg=10             \
                        flug       fldg      fnetg       flug       fldg   
pressure    level                                                          
0.000000    1     -17.779314   0.000000 -17.779314 -17.459490   0.000000   
0.000624    2     -17.779655   0.006016 -17.773639 -17.459848   0.003261   
0.000876    3     -17.779811   0.007392 -17.772419 -17.459994   0.004426   
0.001229    4     -17.780008   0.008970 -17.771038 -17.460185   0.005953   
0.001723    5     -17.780256   0.010752 -17.769504 -17.460433   0.007894   
0.002417    6     -17.780569   0.012747 -17.767822 -17.460757   0.010265   
0.003391    7     -17.780969   0.014979 -17.765990 -17.461182   0.013020   
0.004757    8     -17.781482   0.017490 -17.763992 -17.461741   0.016045   
0.006672    9     -17.782148   0.020351 -17.761797 -17.462483   0.019211   
0.009359    10    -17.783026   0.023658 -17.759368 -17.463477   0.022488   
0.013128    11    -17.784201   0.027538 -17.756663 -17.464825   0.026076   
0.018415    12    -17.785805   0.032149 -17.753656 -17.466682   0.030398   
0.025830    13    -17.788054   0.037677 -17.750377 -17.469290   0.035953   
0.036232    14    -17.791335   0.044337 -17.746999 -17.473066   0.043164   
0.050823    15    -17.796754   0.052409 -17.744346 -17.479110   0.052354   
0.071291    16    -17.806083   0.063244 -17.742839 -17.489109   0.064649   
0.100000    17    -17.819859   0.080451 -17.739408 -17.503538   0.083072   
0.140271    18    -17.837769   0.107797 -17.729971 -17.522101   0.110679   
0.196760    19    -17.860581   0.148492 -17.712089 -17.545488   0.149664   
0.275997    20    -17.888973   0.208339 -17.680634 -17.574232   0.205030   
0.387100    21    -17.922659   0.296436 -17.626224 -17.607749   0.287501   
0.543100    22    -17.958270   0.426827 -17.531443 -17.642083   0.415487   
0.761700    23    -17.978681   0.619912 -17.358769 -17.660000   0.612824   
1.068500    24    -17.941817   0.888772 -17.053045 -17.622902   0.888735   
1.498800    25    -17.834572   1.195008 -16.639564 -17.519291   1.189022   
2.102400    26    -17.688006   1.498725 -16.189282 -17.377245   1.463121   
2.949000    27    -17.512420   1.815988 -15.696432 -17.205368   1.763264   
4.136600    28    -17.309235   2.166257 -15.142978 -17.004450   2.149533   
5.802500    29    -17.080558   2.569448 -14.511110 -16.775650   2.606361   
8.139200    30    -16.822858   3.050440 -13.772418 -16.512917   3.083803   
11.417000   31    -16.527399   3.626315 -12.901084 -16.204566   3.625006   
16.014700   32    -16.200522   4.292234 -11.908288 -15.857870   4.377893   
22.464000   33    -15.884825   5.034963 -10.849862 -15.519514   5.342342   
31.510500   34    -15.603034   5.879610  -9.723424 -15.210580   6.331863   
44.200100   35    -15.321586   6.836779  -8.484808 -14.892281   7.244371   
62.000000   36    -15.069403   7.799571  -7.269832 -14.595010   8.250403   
85.775000   37    -14.997598   8.623551  -6.374047 -14.482950   9.347856   
109.550000  38    -15.099057   9.223570  -5.875487 -14.548624  10.108585   
133.325000  39    -15.263683   9.759378  -5.504305 -14.672748  10.621851   
157.100000  40    -15.604738  10.192260  -5.412478 -14.986270  10.924451   
180.875000  41    -16.352852  10.657536  -5.695316 -15.750701  11.243401   
204.650000  42    -17.440965  11.475226  -5.965739 -16.890629  11.972729   
228.425000  43    -18.588967  12.689898  -5.899069 -18.090814  13.163064   
252.200000  44    -19.724622  14.039158  -5.685464 -19.271767  14.502045   
275.975000  45    -20.842292  15.415922  -5.426370 -20.428811  15.860830   
299.750000  46    -21.940873  16.790817  -5.150056 -21.561968  17.214183   
323.525000  47    -23.019646  18.150446  -4.869201 -22.671421  18.556949   
347.300000  48    -24.077740  19.487061  -4.590679 -23.757007  19.886171   
371.075000  49    -25.113888  20.795616  -4.318271 -24.818018  21.197778   
394.850000  50    -26.125910  22.072350  -4.053560 -25.852657  22.486466 

In [597]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')